In [24]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib3
from tqdm import tqdm
import time

In [27]:
def c_map(number):
    headers ={
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36 OPR/40.0.2308.81',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'DNT': '1',
    'Accept-Encoding': 'gzip, deflate, lzma, sdch',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4'
    }

    URL_TEMPLATE = "https://publichnaya-kadastrovaya-karta.com/object?id=" + number
    r = requests.get(URL_TEMPLATE, headers=headers)
    if r.status_code == 200:
        soup = bs(r.content)
        info = soup.find_all('script')
        coor = str(info).split("var center = L.latLng(")[1].split(");")[0].split(', ')
        return coor
    else:
        print ("Ошибка. Статус:", r.status_code)
        return 0

In [5]:
geo = pd.read_excel('geocoder_legal_entities.xlsx', index_col = 0)
geo.head()

,uindex,latitude,longitude,new_adress
0,11,55.744576,49.283086,"420129 Республика Татарстан, Казань г, 2-я Цен..."
1,12,55.595989,50.386657,"422669 Республика Татарстан, Рыбно-Слободский ..."
2,13,55.691115,51.397837,"Республика Татарстан, Мамадышский р-н, Мамадыш..."
3,14,55.667401,52.339082,"423800 Республика Татарстан, Набережные Челны ..."
4,15,55.659845,52.347347,"423800 Республика Татарстан, Набережные Челны ..."


In [28]:
indexes = geo.index
for i in indexes:
    address = geo['new_adress'][i]
    if address.count(':') > 1:
        if '№' in address:
            cad_num = address.split('№')[1]
            if cad_num.replace(':', '').isnumeric() == False:
                if ',' in cad_num:
                    cad_num = cad_num.split(',')[0]
            if ')' in cad_num:
                cad_num = cad_num.replace(')', '')
            if ';' in cad_num:
                cad_num = cad_num.split(';')[0]
            if ' ' in cad_num:
                cad_num = cad_num.split(' ')[0]
            if '-' in cad_num:
                cad_num = cad_num.split('-')[0]
        elif '(' in address and address.split('(')[1].replace(')', '').replace(':', '').isnumeric():
            cad_num = address.split('(')[1].replace(')', '').replace(':', '')
        
        print(c_map(cad_num))
        time.sleep(3)

ProxyError: HTTPSConnectionPool(host='publichnaya-kadastrovaya-karta.com', port=443): Max retries exceeded with url: /object?id=16:26:340240 (Caused by ProxyError('Your proxy appears to only use HTTP and not HTTPS, try changing your proxy URL to be HTTP. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#https-proxy-error-http-proxy', SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)'))))